# PSI (Population Stability Index) & CSI (Characteristic Stability Index)

## Справка

**PSI** - позволяет измерить, насколько изменилось распределение переменной с течением времени относительно результатов на базовой выборке.

**CSI** - отвечает на вопрос, какая именно переменная вызывает изменения в распределении популяции. По сути - расчет PSI для отдельной фичи.


---



**Алгоритм расчета PSI(CSI):**

1. Разбиваем базовую выборку по выбранному параметру на бины
2. Считаем долю наблюдений в каждом бине на базовой выборке
3. Считаем долю наблюдений в каждом бине на текущей выборке
4. Вычисляем значение индекса PSI по формуле ниже: 

$$PSI = \sum\limits_{bins}\left((\%Actual - \%Expected) \cdot \ln{\frac{\%Actual}{\%Expected}}\right)$$

---

**Интерпретация PSI:**


*   **PSI < 0.1:** в текущей выборке отсутствуют значимые изменения
*   **0.1 < PSI < 0.25:** есть незначительные измения, которые необходимо исследовать
*   **0.25 < PSI:** значительное смещение популяции, требуется перестроение модели


 ---


**Методы разбиения:**
*  По равным бинам - лучше отслеживает изменения в хвостах распределений
*  По перцентилям - лучше отслеживает изменения на вершине "горы"

## Импорты и загрузка синтетических данных

In [ ]:
! pip install nannyml

In [2]:
import numpy as np
import pandas as pd
import datetime
import nannyml as nml # Для загрузки синтетических данных
import plotly.express as px
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

### Синтетические данные (Car Loan Dataset):

---


**reference**: обучающая выборка + таргет + предсказанная вероятность и метка

**analysis**: тестовая выборка + предсказанная вероятность и метка

**analysis_target**: таргет тестовой выборки

In [3]:
# Исходные данные, тестовые данные и тестовые предсказания
reference, analysis, analysis_target = nml.load_synthetic_car_loan_dataset()

# Добавляем пропуски
analysis['repaid_loan_on_prev_car'] = analysis['repaid_loan_on_prev_car'].sample(frac=0.85)
analysis['debt_to_income_ratio'] = analysis['debt_to_income_ratio'].sample(frac=0.85)

display(reference.head())
display(analysis.head())

,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1
3,22652.0,20K - 20K €,0.705992,16.0,False,10%,0.453649,1.0,2018-01-01 00:26:09.456,0.98,1
4,21268.0,60K+ €,0.671888,21.0,True,30%,5.695263,1.0,2018-01-01 00:34:52.608,0.99,1


,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,timestamp,y_pred_proba,y_pred
0,12638.0,0 - 20K €,0.487926,21.0,False,10%,4.224628,2018-10-30 18:00:00.000,0.99,1
1,52425.0,20K - 20K €,0.672183,20.0,False,40%,4.963103,2018-10-30 18:08:43.152,0.98,1
2,20369.0,40K - 60K €,0.703090,19.0,True,40%,4.588951,2018-10-30 18:17:26.304,0.98,1
3,10592.0,20K - 20K €,0.653258,21.0,False,10%,4.711015,2018-10-30 18:26:09.456,0.97,1
4,33933.0,0 - 20K €,0.722263,18.0,False,0%,0.906738,2018-10-30 18:34:52.608,0.92,1


## PSI

In [4]:
def get_cat_buckets(expected_data, actual_data):
    
    """
    Разделяет выборки с дискретными значениями на n_buckets бакетов.
    В случае, когда уникальных числовых значений в выборках меньше, чем n_buckets, вызывает get_cat_buckets()

    Returns:
        buckets - границы бакетов
        expected_bucket_pts - высота каждого бакета в исходной выборке
        actual_bucket_pts - высота каждого бакета в текущей выборке
        expected_data_len - число записей в исходной выборке
        actual_data_len - число записей в текущей выборке
        expected_nans - число пропусков в исходной выборке
        actual_nans - число пропусков в текущей выборке
    """
    
    # Приведение к строковому типу для корректной обработки пропусков
    expected_data = np.array(expected_data).astype(str)
    actual_data = np.array(actual_data).astype(str)

    # Обработка пропусков
    expected_nans = np.count_nonzero(expected_data == 'nan')
    actual_nans = np.count_nonzero(actual_data == 'nan')
    expected_data = expected_data[expected_data != 'nan']
    actual_data = actual_data[actual_data != 'nan']

    # Подсчет количества уникальных значений
    expected_vals, expected_vals_cnt = np.unique(expected_data, return_counts=True)
    actual_vals, actual_vals_cnt = np.unique(actual_data, return_counts=True)
    
    # Объединение уникальных значений из двух выборок
    buckets = np.array(list(set(actual_vals) | set(expected_vals)))

    expected_vals_dict = dict(zip(expected_vals, expected_vals_cnt))
    actual_vals_dict = dict(zip(actual_vals,  actual_vals_cnt))
    
    # 'Заполняем' бакеты
    expected_bucket_pts = np.array([expected_vals_dict.get(val) if val in expected_vals else 0 for val in buckets])
    actual_bucket_pts = np.array([actual_vals_dict.get(val) if val in actual_vals else 0 for val in buckets])
    expected_data_len = np.sum(expected_vals_cnt)
    actual_data_len = np.sum(actual_vals_cnt)

    return buckets, expected_bucket_pts, actual_bucket_pts, expected_data_len, actual_data_len, expected_nans, actual_nans


def get_num_buckets(expected_data, actual_data, n_buckets=10, bucket_type='bins'):
    
    """
    Разделяет числовые выборки на n_buckets бакетов в зависимости от выбранного bucket_type.
    В случае, когда уникальных числовых значений в выборках меньше, чем n_buckets, вызывается get_cat_buckets()

    Returns:
        buckets - границы бакетов
        expected_bucket_pts - высота каждого бакета в исходной выборке
        actual_bucket_pts - высота каждого бакета в текущей выборке
        expected_data_len - число записей в исходной выборке
        actual_data_len - число записей в текущей выборке
        expected_nans - число пропусков в исходной выборке
        actual_nans - число пропусков в текущей выборке
    """

    # Обработка пропусков
    expected_nans = np.count_nonzero(np.isnan(expected_data))
    actual_nans = np.count_nonzero(np.isnan(actual_data))
    expected_data = expected_data[~np.isnan(expected_data)]
    actual_data = actual_data[~np.isnan(actual_data)]
    
    # Подсчет количества уникальных значений в исходных данных
    expected_vals, expected_vals_cnt = np.unique(expected_data, return_counts=True)
    unique_cnt = expected_vals.shape[0]
    
    # Если уникальных значений меньше чем бакетов, то рассчитываем, как для дискретных значений
    if unique_cnt < n_buckets:
        return get_cat_buckets(expected_data, actual_data)

    buckets = np.linspace(0, 1, n_buckets + 1)
    
    # Бакеты равной ширины
    if bucket_type == 'bins':
        unique_vals = np.append(expected_vals, [np.min(actual_data), np.max(actual_data)])
        buckets = np.min(unique_vals) + buckets * (np.max(expected_vals) - np.min(expected_vals))
        
    # Бакеты равной высоты
    elif bucket_type == 'percentiles':
        buckets = np.quantile(expected_data, buckets, method='averaged_inverted_cdf')

    # 'Заполняем' бакеты
    expected_bucket_pts = np.histogram(expected_data, buckets)[0]
    actual_bucket_pts = np.histogram(actual_data, buckets)[0]
    expected_data_len = len(expected_data)
    actual_data_len = len(actual_data)

    # Округление бакетов
    buckets = [round(x, 2) for x in buckets]

    return buckets, expected_bucket_pts, actual_bucket_pts, expected_data_len, actual_data_len, expected_nans, actual_nans


def get_buckets(expected_data, actual_data, n_buckets=10, bucket_type='bins', nan_bucket=True):
    
    """
    Разделяет выборки на бакеты и считает для каждого бакета долю вхождений

    Returns:
        buckets - метки(границы) бакетов
        expected_prc - доля значений в каждом бакете для исходной выборки
        actual_prc - доля значений в каждом бакете для текущей выборки
    """
    
    # Числовые переменные
    if np.issubdtype(np.hstack([expected_data, actual_data]).dtype, np.number):
        buckets, expected_bucket_pts, actual_bucket_pts, expected_data_len, actual_data_len, expected_nans, actual_nans = get_num_buckets(expected_data, actual_data, n_buckets, bucket_type)
    
    # Категориальные переменные
    else:
        buckets, expected_bucket_pts, actual_bucket_pts, expected_data_len, actual_data_len, expected_nans, actual_nans = get_cat_buckets(expected_data, actual_data)

    # Добавление бакета для пропущенных значений
    if nan_bucket and (expected_nans != 0 or actual_nans != 0):
        buckets = np.append(buckets, 'nan')
        expected_bucket_pts = np.append(expected_bucket_pts, expected_nans)
        actual_bucket_pts = np.append(actual_bucket_pts, actual_nans)
        expected_data_len = expected_data_len + expected_nans
        actual_data_len = actual_data_len + actual_nans

    # Расчет процентов
    expected_prc = expected_bucket_pts / expected_data_len
    actual_prc = actual_bucket_pts / actual_data_len

    return buckets, expected_prc, actual_prc


def get_psi(expected_data, actual_data, n_buckets=10, bucket_type='bins', nan_bucket=True):
    """
        Рассчитывает индекс PSI для одной переменной(признака)
        
        Input: 
            expected_data - массив исходных данных
            actual_data - массив текущих данных
            n_buckets - число корзин, на которые делится спектр значений. Не учитывает корзину для пропусков. Не используется при категориальных данных.
            bucket_type - метод разбиения на корзины. 'bins' для корзин равной ширины, 'percentiles' для корзин равной высоты. Не используется при категориальных данных.
            nan_bucket - флаг учета корзины для пропусков
        Output:
            psi - значение индекса
    """
    
    # Поправка для пустых корзин
    EPS = 1e-4

    buckets, expected_prc, actual_prc  = get_buckets(expected_data, actual_data, n_buckets, bucket_type, nan_bucket)
    
    # Обработка пустых корзин
    np.place(expected_prc, expected_prc == 0, EPS)
    np.place(actual_prc, actual_prc == 0, EPS)

    # Расчет индекса
    psi_buckets = (actual_prc - expected_prc) * np.log(actual_prc / expected_prc)
    psi = np.sum(psi_buckets)
    return psi

def get_features_psi(expected_df, actual_df, columns=None, n_buckets=10, bucket_types='bins', nan_buckets=True, returned=False):
    """
        Расчет PSI для всех(или указанных) признаков в датафрейме. Не рассчитывается для столбцов с датами
        Input: 
            expected_df - датафрейм исходных данных
            actual_df - датафрейм текущих данных
            columns - список названий столбцов в исходном датафрейме, которые необходимо учесть. По умолчанию - все
            n_buckets - массив содержащий требуемое число корзин для каждого признака из columns. Не учитывает корзину для пропусков. Не используется при категориальных данных.
            bucket_types - массив содержащий требуемый метод разбиения на корзины для каждого признака из columns. 'bins' для корзин равной ширины, 'percentiles' для корзин равной высоты. Не используется при категориальных данных.
            nan_buckets - bool/массив флагов учета корзины для пропусков для каждого признака из columns
            returned - флаг возвращения списка названий столбцов. Наиболее полезен при columns = None на входе 
        Output:
            features_psi - массив значений индекса для каждого признака
            [columns] - когда returned = True, возвращает названия столбцов, для которых рассчитан индекс
    """
    
    # Обработка пустого списка названий столбцов
    if columns is None:
        columns = expected_df.columns

    features_psi = []

    for i, col in enumerate(columns):
        
        # Обработка столбцов не входящих в исходную выборку
        if col not in expected_df.columns:
            features_psi.append('Not in expected df')

        # Обработка столбцов не входящих в актуальную выборку
        elif col not in actual_df.columns:
            features_psi.append('Not in actual df')
        else:
            
            # Проверка наличия списка n_buckets на входе
            if hasattr(n_buckets, "__len__"):
                n = n_buckets[i]
            else:
                n = n_buckets

            # Проверка наличия списка bucket_types на входе
            if hasattr(bucket_types, "__len__") and type(bucket_types) is not str:
                bucket_type = bucket_types[i]
            else:
                bucket_type = bucket_types

            # Проверка наличия списка nan_buckets на входе
            if hasattr(nan_buckets, "__len__"):
                nan_bucket = nan_buckets[i]
            else:
                nan_bucket = nan_buckets

            psi = get_psi(expected_df[col], actual_df[col], n, bucket_type, nan_bucket)
            features_psi.append(psi)
        
    if returned:
        return features_psi, columns 
    else:
        return features_psi

def get_weighted_psi(expected_df, actual_df, columns=None, weights=None, n_buckets=10, bucket_types='bins', nan_buckets=True, returned=False):
    """
        Расчет PSI методом взвешенного суммирования значений PSI для признаков
        Input: 
            expected_df - датафрейм исходных данных
            actual_df - датафрейм текущих данных
            weights - массив весов для признаков
            columns - список названий колонок в датафрейме, которые необходимо учесть в суммировании. По умолчанию - все
            n_buckets - массив содержащий требуемое число корзин для каждого признака. Не учитывает корзину для пропусков. Не используется при категориальных данных.
            bucket_types - массив содержащий требуемый метод разбиения на корзины для каждого признака. 'bins' для корзин равной ширины, 'percentiles' для корзин равной высоты. Не используется при категориальных данных.
            nan_buckets - bool/массив флагов учета корзины для пропусков
        Output:
            weighted_psi - значение индекса
            [sum_of_weights] - когда returned = True, возвращает сумму весов
    """

    # Обработка пустого списка названий столбцов
    if columns is None:
        columns = expected_df.columns
        
    # Расчет PSI для признаков из списка
    features_psi = get_features_psi(expected_df, actual_df, columns, n_buckets, bucket_types, nan_buckets)

    # Маска корректных значений PSI
    mask = [type(feature_psi) is not str for feature_psi in features_psi]

    # Применение маски к массиву PSI для признаков
    features_psi_masked = np.array(features_psi)[mask]
    features_psi_masked = features_psi_masked.astype(np.float64)
    
    # Обработка пустого списка весов и применение маски
    weights_masked = weights
    if weights is not None:
        weights_masked = np.array(weights)[mask]
    
    # Расчет взвешенной суммы
    weighted_psi = np.average(features_psi_masked, weights=weights_masked, returned=returned)

    return weighted_psi


## Примеры

PSI предсказанных меток с бакетами одинаковой ширины

In [5]:
buckets, expected_prc, actual_prc  = get_buckets(reference.y_pred_proba, analysis.y_pred_proba, n_buckets=15, bucket_type='bins')
psi = get_psi(reference.y_pred_proba, analysis.y_pred_proba, n_buckets=15, bucket_type='bins')

print('PSI: ', psi, '\n')
print('Buckets borders: ', buckets, '\n')
print('Expected percents in buckets: ', expected_prc, '\n')
print('Actual percents in buckets: ', actual_prc, '\n')

PSI:  0.0667574535226368 

Buckets borders:  [0.0, 0.07, 0.13, 0.2, 0.27, 0.33, 0.4, 0.47, 0.53, 0.6, 0.67, 0.73, 0.8, 0.87, 0.93, 1.0] 

Expected percents in buckets:  [0.3585  0.07358 0.02172 0.01476 0.00968 0.00784 0.00802 0.00762 0.00638
 0.01008 0.01306 0.01504 0.02792 0.07956 0.34624] 

Actual percents in buckets:  [0.29412 0.08692 0.03344 0.0243  0.01698 0.01236 0.01292 0.01122 0.01062
 0.0148  0.01902 0.02272 0.0452  0.10206 0.29332] 



PSI предсказанных меток с бакетами одинаковой высоты

In [6]:
buckets, expected_prc, actual_prc  = get_buckets(reference.y_pred_proba, analysis.y_pred_proba, n_buckets=15, bucket_type='percentiles')
psi = get_psi(reference.y_pred_proba, analysis.y_pred_proba, n_buckets=15, bucket_type='percentiles')

print('PSI: ', psi, '\n')
print('Buckets borders: ', buckets, '\n')
print('Expected percents in buckets: ', expected_prc, '\n')
print('Actual percents in buckets: ', actual_prc, '\n')

PSI:  0.07423276503140393 

Buckets borders:  [0.0, 0.0, 0.01, 0.02, 0.03, 0.05, 0.09, 0.25, 0.75, 0.9, 0.94, 0.96, 0.98, 0.99, 0.99, 1.0] 

Expected percents in buckets:  [0.      0.07606 0.07858 0.06528 0.08768 0.08368 0.07402 0.06802 0.06414
 0.0563  0.0536  0.089   0.06472 0.      0.13892] 

Actual percents in buckets:  [0.      0.0541  0.06098 0.05164 0.07702 0.0855  0.10364 0.10716 0.09768
 0.06896 0.05342 0.08394 0.05212 0.      0.10384] 



PSI категориального признака

In [7]:
buckets, expected_prc, actual_prc  = get_buckets(reference.salary_range, analysis.salary_range)
psi = get_psi(reference.salary_range, analysis.salary_range)

print('PSI: ', psi, '\n')
print('Buckets: ', buckets, '\n')
print('Expected percents in buckets: ', expected_prc, '\n')
print('Actual percents in buckets: ', actual_prc, '\n')

PSI:  0.03189615940796424 

Buckets:  ['20K - 20K €' '40K - 60K €' '0 - 20K €' '60K+ €'] 

Expected percents in buckets:  [0.30094 0.19954 0.39836 0.10116] 

Actual percents in buckets:  [0.3153  0.21032 0.4203  0.05408] 



PSI с бакетом для пропусков

In [8]:
buckets, expected_prc, actual_prc  = get_buckets(reference.repaid_loan_on_prev_car, analysis.repaid_loan_on_prev_car)
psi = get_psi(reference.repaid_loan_on_prev_car, analysis.repaid_loan_on_prev_car)

print('PSI: ', psi, '\n')
print('Buckets: ', buckets, '\n')
print('Expected percents in buckets: ', expected_prc, '\n')
print('Actual percents in buckets: ', actual_prc, '\n')

PSI:  1.1815308464315968 

Buckets:  ['True' 'False' 'nan'] 

Expected percents in buckets:  [0.55094 0.44906 0.     ] 

Actual percents in buckets:  [0.57408 0.27592 0.15   ] 



Тот же PSI без бакета для пропусков

In [9]:
buckets, expected_prc, actual_prc  = get_buckets(reference.repaid_loan_on_prev_car, analysis.repaid_loan_on_prev_car, nan_bucket=False)
psi = get_psi(reference.repaid_loan_on_prev_car, analysis.repaid_loan_on_prev_car, nan_bucket=False)

print('PSI: ', psi, '\n')
print('Buckets: ', buckets, '\n')
print('Expected percents in buckets: ', expected_prc, '\n')
print('Actual percents in buckets: ', actual_prc, '\n')

PSI:  0.0657321129890791 

Buckets:  ['True' 'False'] 

Expected percents in buckets:  [0.55094 0.44906] 

Actual percents in buckets:  [0.67538824 0.32461176] 



PSI для всех (или указанных) столбцов в датафрейме

In [10]:
columns = ['car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure', 'repaid']
features_psi = get_features_psi(reference, analysis, columns=columns)
feature_psi_df = pd.DataFrame(np.column_stack(features_psi), columns=columns, index=[0])
display(feature_psi_df)

,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid
0,0.23185772351705353,0.03189615940796424,1.1210321903793046,0.05520377254691115,1.1815308464315968,0.0001368318789760221,0.00035966116429024216,Not in actual df


Средневзвешенный PSI для весов по умолчанию

In [11]:
columns = ['car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure', 'repaid']
weighted_psi = get_weighted_psi(reference, analysis, columns=columns, returned=True)
print('Weighted PSI: ', weighted_psi[0], '\nSum of weights: ', weighted_psi[1])

Weighted PSI:  0.3745738836180138 
Sum of weights:  7.0


Средневзвешенный PSI для заданных весов

In [12]:
weights = np.random.random(len(columns))
print('Weights: ', weights)

weighted_psi = get_weighted_psi(reference, analysis, columns=columns, weights=weights, returned=True)
print('Weighted PSI: ', weighted_psi[0], '\nSum of weights: ', weighted_psi[1])

Weights:  [0.40901498 0.43548808 0.25609914 0.05008591 0.46674084 0.04547872
 0.37524393 0.79501583]
Weighted PSI:  0.46620372580599323 
Sum of weights:  2.038151596019368


## Визуализация

In [17]:
expected_data = reference.y_pred_proba.copy()
actual_data_by_dates = analysis[['timestamp', 'y_pred_proba']].copy()

expected_data.columns = ['y']
actual_data_by_dates.columns = ['date', 'y']

actual_data_by_dates.date = actual_data_by_dates.date.str[:7]

n_buckets=15
bucket_type='bins'
plot=True
nan_bucket = False

expected_plot = []
actual_plot = []

psi_by_dates = []



# считаем psi для каждого месяца из actual
for date in actual_data_by_dates.date.unique():
    actual_data = actual_data_by_dates[actual_data_by_dates.date == date].y.copy()

    # проценты expected и actual в каждой корзине
    bucket_borders, expected_prc, actual_prc = get_buckets(expected_data, actual_data, n_buckets, bucket_type, nan_bucket)
    

    # сохраняем списки в один плоский список
    expected_plot = np.hstack([expected_plot, expected_prc])
    actual_plot = np.hstack([actual_plot, actual_prc])

    # расчет PSI
    psi = get_psi(expected_data, actual_data, n_buckets, bucket_type, nan_bucket)

    psi_by_dates.append(psi)



In [18]:
if plot:
    bucket_borders_str = [f"{bucket_borders[i]} - {bucket_borders[i + 1]}" for i in range(n_buckets)]

    date = [date for date in actual_data_by_dates.date.unique() for j in range(n_buckets)]
    bucket = [bucket_name for elem in actual_data_by_dates.date.unique() for bucket_name in bucket_borders_str]

    freq_df = pd.DataFrame({'Месяц': date, 'Границы бакетов': bucket, 'score_expected': expected_plot, 'score_actual': actual_plot})
    df_long = pd.wide_to_long(freq_df, stubnames='score', i=['Месяц', 'Границы бакетов'], j=' ', sep='_', suffix='\w+').reset_index()

    fig = px.bar(df_long, x='Границы бакетов', y = 'score', 
                 hover_name = "Месяц", color=' ', 
                 animation_frame= 'Месяц', barmode='group',
                 range_y=[0, df_long['score'].max()],
                title="Изменение распределения скора модели говорит об ухудшении/улучшении потока клиентов")

    fig.update_layout(
        width=1000, height=500, 
        template='plotly_white',
        legend=dict(x=0,y=1, traceorder='normal', font=dict(size=12,),),
    )
    fig.show()

print('Actual dates: ', actual_data_by_dates.date.unique(), '\n')
print('PSI by date: ', psi_by_dates)

Actual dates:  ['2018-10' '2018-11' '2018-12' '2019-01' '2019-02' '2019-03' '2019-04'
 '2019-05' '2019-06' '2019-07' '2019-08'] 

PSI by date:  [0.15684973946799904, 0.0030748875997473404, 0.0023781652740934413, 0.002995573324074608, 0.0027704542205133557, 0.002958640557645117, 0.27019988710167786, 0.24354824111930182, 0.2705986789166045, 0.22695283328057403, 0.24760581324065273]
